In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [5]:
import torch
import torch.nn as nn

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32

Using cuda device


In [6]:
x0 = 1.0 # initial condition
sigma = 1 # volatility
mu = 0.02
segs=20
r = 0.01 # risk free rate
batch_size = 1000 # batch size
steps=5000
T = 1 # maturity
dt = T/steps # mesh size
true = 0.5828174603130847# true option price

dt = T/steps # mesh size
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level

In [17]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [8]:
def create_stock2(x0,r,sigma,T,steps,n_path,dW):
    dt=T/steps; 
    sqrt_dt=np.sqrt(dt); 
    s_vec=[]; w_vec=[]; 
    w_vec.append(np.ones(n_path)*1e-6)
    s_vec.append(np.ones(n_path)*1.0)
    for i in range(steps): 
        w_vec.append(dW[:,i]) 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ w_vec[-1]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    wvec=np.transpose(np.array(w_vec))
    BM_path=np.cumsum(wvec,axis=1)
    S_path=np.transpose(np.array(s_vec))
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt)
    times=np.append(times,T); 
    times_vec=np.tile(times,[2,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec[:,:,1]=path
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth):
    n_batch, nsteps,n_path = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    
   # path_class=signatory.Path(path,depth);
    ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
    MultiLevelSig.append(ll)
    
    for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
        MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
        #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
    MultiLevelSig=np.stack(MultiLevelSig)  
    MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    return MultiLevelSig

In [9]:
def generate_samples(batch_in=100):
    dW = np.sqrt(dt)*np.random.normal(size=(batch_in, steps))
    pth2=create_stock2(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1]
    dW=BM_seg[:,1:]-BM_seg[:,:-1]
    dW=np.expand_dims(dW,axis=2)

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)

    ss=S_timePath[:,:,1]
    YT=ss[:,-1]-np.min(ss,axis=1)
    YT=torch.tensor(YT,dtype=torch.float32)
    YT=YT.unsqueeze(axis=1)
    return sigs, dW, YT

In [10]:
class Lookback_PPDE(nn.Module):
    def __init__(self): 
        super(Lookback_PPDE,self).__init__()
        self.Y0=Parameter(torch.rand(100,1))
        self.model=RNN2(input_size=14, output_size=1, hidden_size=128, num_layers=2)
             
    def forward(self,batch_sig,batch_dW): 
        Z_path=self.model(batch_sig)
        Y=self.Y0
        for i in range(segs):
            Y=Y+Y*r*T/segs+sigma*Z_path[:,i,:]*batch_dW[:,i,:]
        return Y

In [37]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=14; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,128,64,1] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [59]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
    #    self.Y0=Parameter(torch.tensor([0.0])) #torch.rand(500,1)
        self.Y0=Parameter(torch.rand(100,1))
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW): 
   #     Z_path=self.model(batch_sig)
        Y=self.Y0
        for i in range(segs):
            Y=Y+Y*r*T/segs+sigma*self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:]
        return Y

In [60]:
def Loss(Y_pred,Y_true):
    res=torch.mean((Y_pred-Y_true)**2)
    return res

In [61]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE=Lookback_PPDE(cfg)
optimizer=optim.Adam(model_PPDE.parameters(),lr=1e-3)
epochs=400 

In [62]:
def generate_samples(batch_in=100):
    dW = np.sqrt(dt)*np.random.normal(size=(batch_in, steps))
    pth2=create_stock2(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1]
    dW=BM_seg[:,1:]-BM_seg[:,:-1]
    dW=np.expand_dims(dW,axis=2)

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)

    ss=S_timePath[:,:,1]
    YT=ss[:,-1]-np.min(ss,axis=1)
    YT=torch.tensor(YT,dtype=torch.float32)
    YT=YT.unsqueeze(axis=1)
    return sigs, dW, YT

In [63]:
model_PPDE=Lookback_PPDE(cfg)
model_PPDE
optimizer=optim.Adam(model_PPDE.parameters(),lr=2e-3)
grad_clip=0.2
y0_mean=[];
loss_vec=[];

In [64]:
for i in range(2500):
    batch_x, batch_dw, batch_y =generate_samples(batch_in=100)
    

    x_temp=model_PPDE(batch_x,batch_dw)
    loss_temp=Loss(x_temp, batch_y)
    
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    
    y0_val=model_PPDE.Y0.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)

    if i%10==0:
      #  print("Iter:", i,  loss_temp) 
        print("Iter:", i,  model_PPDE.Y0.mean(), loss_temp)

Iter: 0 tensor(0.4824, grad_fn=<MeanBackward0>) tensor(1.5609, grad_fn=<MeanBackward0>)
Iter: 10 tensor(0.4820, grad_fn=<MeanBackward0>) tensor(1.1977, grad_fn=<MeanBackward0>)
Iter: 20 tensor(0.4848, grad_fn=<MeanBackward0>) tensor(0.1789, grad_fn=<MeanBackward0>)
Iter: 30 tensor(0.4880, grad_fn=<MeanBackward0>) tensor(0.1655, grad_fn=<MeanBackward0>)
Iter: 40 tensor(0.4913, grad_fn=<MeanBackward0>) tensor(0.1193, grad_fn=<MeanBackward0>)
Iter: 50 tensor(0.4945, grad_fn=<MeanBackward0>) tensor(0.1322, grad_fn=<MeanBackward0>)
Iter: 60 tensor(0.4979, grad_fn=<MeanBackward0>) tensor(0.1365, grad_fn=<MeanBackward0>)
Iter: 70 tensor(0.5011, grad_fn=<MeanBackward0>) tensor(0.1083, grad_fn=<MeanBackward0>)
Iter: 80 tensor(0.5041, grad_fn=<MeanBackward0>) tensor(0.0927, grad_fn=<MeanBackward0>)
Iter: 90 tensor(0.5070, grad_fn=<MeanBackward0>) tensor(0.1731, grad_fn=<MeanBackward0>)
Iter: 100 tensor(0.5095, grad_fn=<MeanBackward0>) tensor(0.0939, grad_fn=<MeanBackward0>)
Iter: 110 tensor(0.51

Iter: 920 tensor(0.5774, grad_fn=<MeanBackward0>) tensor(0.0654, grad_fn=<MeanBackward0>)
Iter: 930 tensor(0.5771, grad_fn=<MeanBackward0>) tensor(0.0735, grad_fn=<MeanBackward0>)
Iter: 940 tensor(0.5768, grad_fn=<MeanBackward0>) tensor(0.1002, grad_fn=<MeanBackward0>)
Iter: 950 tensor(0.5768, grad_fn=<MeanBackward0>) tensor(0.0447, grad_fn=<MeanBackward0>)
Iter: 960 tensor(0.5774, grad_fn=<MeanBackward0>) tensor(0.9063, grad_fn=<MeanBackward0>)
Iter: 970 tensor(0.5775, grad_fn=<MeanBackward0>) tensor(0.0629, grad_fn=<MeanBackward0>)
Iter: 980 tensor(0.5775, grad_fn=<MeanBackward0>) tensor(0.0738, grad_fn=<MeanBackward0>)
Iter: 990 tensor(0.5772, grad_fn=<MeanBackward0>) tensor(0.0421, grad_fn=<MeanBackward0>)
Iter: 1000 tensor(0.5768, grad_fn=<MeanBackward0>) tensor(0.3633, grad_fn=<MeanBackward0>)
Iter: 1010 tensor(0.5772, grad_fn=<MeanBackward0>) tensor(0.0952, grad_fn=<MeanBackward0>)
Iter: 1020 tensor(0.5775, grad_fn=<MeanBackward0>) tensor(0.0883, grad_fn=<MeanBackward0>)
Iter: 1

Iter: 1830 tensor(0.5817, grad_fn=<MeanBackward0>) tensor(0.1058, grad_fn=<MeanBackward0>)
Iter: 1840 tensor(0.5818, grad_fn=<MeanBackward0>) tensor(0.1070, grad_fn=<MeanBackward0>)
Iter: 1850 tensor(0.5814, grad_fn=<MeanBackward0>) tensor(0.1083, grad_fn=<MeanBackward0>)
Iter: 1860 tensor(0.5809, grad_fn=<MeanBackward0>) tensor(0.0914, grad_fn=<MeanBackward0>)
Iter: 1870 tensor(0.5806, grad_fn=<MeanBackward0>) tensor(0.0631, grad_fn=<MeanBackward0>)
Iter: 1880 tensor(0.5801, grad_fn=<MeanBackward0>) tensor(0.7190, grad_fn=<MeanBackward0>)
Iter: 1890 tensor(0.5806, grad_fn=<MeanBackward0>) tensor(0.0412, grad_fn=<MeanBackward0>)
Iter: 1900 tensor(0.5816, grad_fn=<MeanBackward0>) tensor(0.1014, grad_fn=<MeanBackward0>)
Iter: 1910 tensor(0.5822, grad_fn=<MeanBackward0>) tensor(0.1031, grad_fn=<MeanBackward0>)
Iter: 1920 tensor(0.5830, grad_fn=<MeanBackward0>) tensor(0.1147, grad_fn=<MeanBackward0>)
Iter: 1930 tensor(0.5841, grad_fn=<MeanBackward0>) tensor(0.1739, grad_fn=<MeanBackward0>)

In [66]:
y_pred=np.array(y0_mean)
loss_var=np.array(loss_vec)
iters=np.arange(0,len(y_pred),1)+1

In [68]:
import pandas as pd

In [69]:
df=pd.DataFrame()
df['iter']=iters
df['y_pred']=y_pred
df['loss_var']=loss_var

In [70]:
df.to_csv('trained_data/method1_1.csv')

In [75]:
df['y_pred'][2000:].mean()

0.58113813

In [88]:
df[df.loss_var<=0.1].y_pred[-500:].mean()

0.581476